# Setup

In [ ]:
import numpy as np
import pandas as pd
import ipdb
from collections import defaultdict, Counter
import glob

import sys
import seaborn as sns
import pylab

import matplotlib.pyplot as plt

import geopandas as gpd

PAPER_DATA_TIME = 1600387200 # September 25th, 2020 (approximate)

def print_full(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [ ]:
ISPS_PER_STATE = { # major ISPs
'AR': ['att', 'centurylink', 'xfinity', 'cox', 'windstream'],
'ME': ['consolidated','charter'],
'MA': ['verizon','xfinity','charter'],
'NY': ['verizon', 'charter', 'frontier'],
'NC': ['charter','att','centurylink','windstream', 'frontier'],
'OH': ['att','charter', 'frontier', 'centurylink', 'windstream'],
'VT': ['consolidated', 'xfinity'],
'VA': ['xfinity', 'cox', 'verizon', 'centurylink'],
'WI': ['att','charter', 'centurylink', 'frontier']
}

ISP_FULL_NAMES = {
'windstream': 'Windstream Holdings, Inc.',
'charter': 'Charter Communications',
'xfinity': 'Comcast Corporation',
'centurylink':'CenturyLink, Inc.',
'att': 'AT&T Inc.',
'cox': 'Cox Communications, Inc.',
'verizon': 'Verizon Communications Inc.',
'frontier': 'Frontier Communications Corporation',
'consolidated': 'Consolidated Communications, Inc.',
}

major_isps_rev = {v:k for k, v in ISP_FULL_NAMES.items()}

STATES = ['AR', 'ME', 'MA', 'NY', 'NC', 'OH', 'VT', 'VA', 'WI']
STATES_TUPLE = ('AR', 'ME', 'MA', 'NY', 'NC', 'OH','VT','VA','WI')

# Positive responses
POS_RESPONSES = {
'att': ('2', '3'),
'xfinity': ('2','3'),
'centurylink': ('1',),
'cox': ('1',),
'consolidated': ('1',),
'verizon': ('1','25'),
'windstream': ('1',),
'charter': ('1', ),
'frontier': ('2','3'), 
}

# Negative responses (no coverage)
NEG_RESPONSES = {
'att': ('0',),
'xfinity': ('0',),
'centurylink': ('20', '21'),
'cox': ('0',),
'consolidated': ('0', '20'),
'verizon': ('0', '21'),
'windstream': ('22','24'),
'charter': ('0','29'),
'frontier':('0','21'), 

}

# Responses that the address is unrecognized
UNRECOGNIZED_RESPONSES = {
'att': ('4',),
'xfinity': ('4',),
'centurylink': ('0','4'),
'cox': ('4',),
'consolidated': ('21','22'),
'verizon': ('4',),
'windstream': ('20','23'),
'charter': (),
'frontier': (), 
}

# Business responses
BUSINESS_RESPONSES = {
'att': (),
'xfinity': ('5',),
'centurylink': (),
'cox': ('5',),
'consolidated': (),
'verizon': (),
'windstream': (),
'charter': (),
'frontier':(), 
}

# Sensitivity Analysis 2 (Table 12)
UNKNOWN_RESPONSES = {
'att': ('23', '24', '25', '26','30', '34'),
'xfinity': ('20', '21', '28', '29', '33', '34'),
'centurylink': ('-60', '-65',  '22', '24', '31'),
'cox': (),
'consolidated': ('23','31','32'),
'verizon': ('23','24','25','30'),
'windstream': ('21',),
'charter': ('22','24','25'),
'frontier': ('22','23'),
}

# Excluded (If not in UNKNOWN_RESPONSES)
EXCLUDED_RESPONSES = {
'att': ('23', '24', '25', '26','30', '34'),
'xfinity': ('20', '21', '28', '29', '33', '34', '40'),
'centurylink': ('-60', '-65',  '22', '24', '31', '33'),
'cox': ('31',),
'consolidated': ('23','31','32'),
'verizon': ('23','24','25','30'),
'windstream': ('21',),
'charter': ('22','24','25','31','50','51','60'),
'frontier': ('22','23'), 
}

ISP_TECH_DATA = {
    'verizon': {'states': STATES, 'tech_codes': {'fcc': [10, 50], 'tool': [10, 50]}},
    'centurylink': {'states': ['AR', 'NC'], 'tech_codes': {'fcc': [10, 50]}},
    'att': {'states': STATES, 'tech_codes': {'fcc': [10, 50, 70], 'tool': [10, 50, 70]}},
    'windstream': {'states': ['NC'], 'tech_codes': {'fcc': [10, 41, 50]}}
}

VALID_STATE_FIPS = {
    5: "AR",
    23: "ME",
    25: "MA",
    36: "NY",
    37: "NC",
    39: "OH",
    50: "VT",
    51: "VA",
    55: "WI"
}

LABELS = {
'att': 'AT&T',
'centurylink': 'CenturyLink',
'consolidated': 'Consolidated',
'cox': 'Cox',
'xfinity' : 'Comcast',
'verizon' : 'Verizon',
'windstream': 'Windstream',
'charter': 'Charter',
'frontier': 'Frontier',
'VA': 'Virginia',
'VT': 'Vermont',
'AR': 'Arkansas',
'MA': 'Massachusetts',
'NC': 'North Carolina',
'NY': 'New York',
'ME': 'Maine',
'OH': 'Ohio',
'WI': 'Wisconsin',
'U': 'Urban',
'R': 'Rural',
'Total': 'Total'
}

SORTED_ISPS = ['att', 'centurylink', 'charter', 'xfinity', 'consolidated', 'cox', 'frontier', 'verizon', 'windstream']

## Load FCC Form 477 Data (Unused)

In [ ]:
form477_data = pd.read_csv("fbd_us_without_satellite_jun2018_v1.csv", encoding='latin1')
form477_data = form477_data[form477_data["Consumer"] == 1]

In [ ]:
if "form477_isps" in globals():
    del form477_isps
if "form477_local" in globals():
    del form477_local
form477_isps = defaultdict(set)
form477_local = defaultdict(set)
for listing in form477_data.itertuples():
    form477_isps[listing.BlockCode].add(listing.HocoFinal)
    if listing.HocoFinal not in major_isps_rev:
        form477_local[listing.BlockCode].add(listing.HocoFinal)
del form477_data

## Load Population Data

In [ ]:
pop_data = pd.read_csv('us2019.csv')

In [ ]:
if "pop_dict" in locals():
    del pop_dict
pop_dict = {}
for block in pop_data.itertuples():
    pop_dict[block.block_fips] = block.pop2018
    
del pop_data

## Load Urban/Rural Data

In [ ]:
urban_rural_dict = {}

In [ ]:
for state in ISPS_PER_STATE:
    print(state)
    shp = gpd.read_file(glob.glob(f"block_class/{state}/*.shp")[0])
    
    data = shp[shp.UR10 == 'R']
    for blockno in data.GEOID10.astype(int).unique():
        urban_rural_dict[blockno] = 'R'
    
    data = shp[shp.UR10 == 'U']
    for blockno in data.GEOID10.astype(int).unique():
        urban_rural_dict[blockno] = 'U'

# Load Dataset

In [ ]:
data_columns = {}

with open('columns.csv') as f:
    curr_state = ""
    curr_columns = []
    for line in f.read().splitlines():
        if not line:
            data_columns[curr_state] = curr_columns
            curr_state = ""
            curr_columns = []
        elif len(line) == 2:
            curr_state = line
        else:
            curr_columns.append(line)

In [ ]:
all_state_data = {}
all_null_data = {}

for state, isps in ISPS_PER_STATE.items():
    print(state)
    
    null_data = []
    
    state_cols = data_columns[state]
    
    state_data = pd.read_csv("data/data_{}.csv".format(state), names=state_cols, header=0)
    state_data = state_data[(state_data['addr_dpv'].isin(['Y', 'D', 'S'])) & (state_data['addr_rdi'] == 'R')] # filter USPS DPV and RDI
    
    state_data["state_tb"] = state
    
    null_data.append(state_data[state_data["addr_census_block"].isna()])
    state_data = state_data[~state_data["addr_census_block"].isna()]
    
    null_data.append(state_data[state_data["fcc_coverage_LOCAL"].isna()])
    state_data = state_data[~state_data["fcc_coverage_LOCAL"].isna()]
    
    state_data["addr_census_block"] = state_data["addr_census_block"].astype(int)
    
    all_state_data[state] = state_data
    all_null_data[state] = pd.concat(null_data, ignore_index=True)

## Build Data Columns

In [ ]:
data_cols = {} # state -> {'isp': {fcc': [cols], 'tool', [cols], 'local': [cols]}}

for state, isps in ISPS_PER_STATE.items():
    state_dict = {}
    for isp in isps:
        fcc_cols = []
        fcc_speed_cols = []
        tool_cols = []
        tool_speed_cols = []
        timestamp_cols = []
        
        has_fcc_code = False
        has_tool_code = False

        if isp in ISP_TECH_DATA:
            isp_tech_data = ISP_TECH_DATA[isp]
            if state in isp_tech_data["states"]:
                codes = isp_tech_data["tech_codes"]
                if "fcc" in codes:
                    has_fcc_code = True
                    fcc_codes = codes["fcc"]
                    for fcc_code in fcc_codes:
                        fcc_cols.append(f"fcc_coverage_{isp}_{fcc_code}")
                        fcc_speed_cols.append(f"fcc_coverage_downspeed_{isp}_{fcc_code}")
                    if "tool" in codes:
                        has_tool_code = True
                        tool_codes = codes["tool"]
                        for tool_code in tool_codes:
                            tool_cols.append(f"tool_coverage_{isp}_{tool_code}")
                            tool_speed_cols.append(f"tool_coverage_downspeed_{isp}_{tool_code}")
                            timestamp_cols.append(f"tool_timestamp_{isp}_{tool_code}")
        
        if not has_fcc_code:
            fcc_cols.append(f"fcc_coverage_{isp}")
            fcc_speed_cols.append(f"fcc_coverage_downspeed_{isp}")
        if not has_tool_code:
            tool_cols.append(f"tool_coverage_{isp}")
            tool_speed_cols.append(f"tool_coverage_downspeed_{isp}")
            timestamp_cols.append(f"tool_timestamp_{isp}")
            
        state_dict[isp] = {
            'fcc': fcc_cols,
            'fcc_speed': fcc_speed_cols,
            'tool': tool_cols,
            'tool_speed': tool_speed_cols,
            'timestamp': timestamp_cols
        }
        
    if state == "NY":
        local_col = "fcc_coverage_LOCAL_altice"
        local_speed_col = "fcc_coverage_downspeed_LOCAL_altice"
    else:
        local_col = "fcc_coverage_LOCAL"
        local_speed_col = "fcc_coverage_downspeed_LOCAL"
    state_dict["local"] = local_col
    state_dict["local_speed"] = local_speed_col
        
    data_cols[state] = state_dict
    
def get_cols(state, typ):
    cols = []
    for isp in data_cols[state].values():
        if "LOCAL" not in isp:
            cols.extend(isp[typ])
    return cols

## Compute Table 5

In [ ]:
# Table 5
table5_dict = {0: {}, 25: {}}

for speed in [0, 25]:
    for state in ISPS_PER_STATE:
        
        state_dict = {}
        table5_dict[speed][state] = state_dict

        state_data = all_state_data[state]
        state_data_cols = data_cols[state]

        for address in state_data.itertuples():
            blockno = getattr(address, "addr_census_block")

            block_data = state_dict.get(blockno, [0, 0])

            local_col = state_data_cols["local"]
            local_speed_col = state_data_cols["local_speed"]
            local_speed_data = getattr(address, local_speed_col)

            has_local_ISP = False
            if getattr(address, local_col) == 1 and local_speed_data >= speed:
                block_data[0] += 1
                block_data[1] += 1
                has_local_ISP = True


            if not has_local_ISP:
                fcc_covered = False
                bat_covered = False
                bad = False # unrecognized/unknown

                for isp in ISPS_PER_STATE[state]:
                    if bat_covered:
                        break

                    isp_fcc_covered = False

                    state_isp_data_cols = state_data_cols[isp]
                    fcc_cols = state_isp_data_cols["fcc"]
                    fcc_speed_cols = state_isp_data_cols["fcc_speed"]
                    tool_cols = state_isp_data_cols["tool"]
                    tool_speed_cols = state_isp_data_cols["tool_speed"]
                    timestamp_cols = state_isp_data_cols["timestamp"]

                    for i, fcc_col in enumerate(fcc_cols):
                        fcc_speed_col = fcc_speed_cols[i]
                        if getattr(address, fcc_col) == 1 and getattr(address, fcc_speed_col) >= speed:
                            isp_fcc_covered = True
                            fcc_covered = True

                    if not isp_fcc_covered:
                        continue

                    for i, tool_col in enumerate(tool_cols):
                        tool_speed_col = tool_speed_cols[i]
                        timestamp_col = timestamp_cols[i]
                        
                        timestamp = getattr(address, timestamp_col)
                        if timestamp > PAPER_TIME:
                            bad = True
                            continue

                        try:
                            tool_data = str(int(getattr(address, tool_col))) # implicitly reports NaNs :D
                        except:
                            bad = True
                            continue
                        
                        if tool_data in POS_RESPONSES[isp]:
                            bat_covered = True
                            break

                        elif tool_data not in NEG_RESPONSES[isp]: # unrecognized/unknown/nan
                            bad = True

                if bat_covered or (fcc_covered and not bad):
                    block_data[0] += 1 # FCC
                    if bat_covered:
                        block_data[1] += 1 # BAT

            state_dict[blockno] = block_data

In [ ]:
table5_results_dict = {0: {}, 25: {}}
for speed in [0, 25]:
    total_fcc_counts = {
        'U': 0,
        'R': 0
    }
    total_bat_counts = {
        'U': 0,
        'R': 0
    }
    total_fcc_populations = {
        'U': 0,
        'R': 0
    }
    total_bat_populations = {
        'U': 0,
        'R': 0
    }
    
    for state in table5_dict[speed]:
        print(state)

        res = {}
        table5_results_dict[speed][state] = res

        fcc_counts = {
            'U': 0,
            'R': 0
        }
        bat_counts = {
            'U': 0,
            'R': 0
        }
        fcc_populations = {
            'U': 0,
            'R': 0
        }
        bat_populations = {
            'U': 0,
            'R': 0
        }

        state_data = table5_dict[speed][state]

        for blockno in state_data:
            block_data = state_data[blockno]
            try:
                block_class = urban_rural_dict[blockno]
            except:
                continue

            if block_data[0] > 0:
                fcc_populations[block_class] += pop_dict[blockno]
                bat_populations[block_class] += pop_dict[blockno] * (block_data[1]/block_data[0])

                fcc_counts[block_class] += block_data[0]
                bat_counts[block_class] += block_data[1]

        res['fcc_count_total'] = fcc_counts['R'] + fcc_counts['U']
        res['fcc_count_U'] = fcc_counts['U']
        res['fcc_count_R'] = fcc_counts['R']

        res['bat_count_total'] = bat_counts['R'] + bat_counts['U']
        res['bat_count_U'] = bat_counts['U']
        res['bat_count_R'] = bat_counts['R']

        res['count_per_total'] = str(100 * res['bat_count_total'] / res['fcc_count_total'])
        res['count_per_U'] = str(100 * res['bat_count_U'] / res['fcc_count_U'])
        res['count_per_R'] = str(100 * res['bat_count_R'] / res['fcc_count_R'])

        res['fcc_pop_total'] = fcc_populations['R'] + fcc_populations['U']
        res['fcc_pop_U'] = fcc_populations['U']
        res['fcc_pop_R'] = fcc_populations['R']
        
        res['bat_pop_total'] = int(bat_populations['R'] + bat_populations['U'])
        res['bat_pop_U'] = int(bat_populations['U'])
        res['bat_pop_R'] = int(bat_populations['R'])
        
        res['pop_per_total'] = str(100 * res['bat_pop_total'] / res['fcc_pop_total'])
        res['pop_per_U'] = str(100 * res['bat_pop_U'] / res['fcc_pop_U'])
        res['pop_per_R'] = str(100 * res['bat_pop_R'] / res['fcc_pop_R'])
        
        total_fcc_counts['U'] += fcc_counts['U']
        total_fcc_counts['R'] += fcc_counts['R']
        total_bat_counts['U'] += bat_counts['U']
        total_bat_counts['R'] += bat_counts['R']
        
        total_fcc_populations['U'] += fcc_populations['U']
        total_fcc_populations['R'] += fcc_populations['R']
        total_bat_populations['U'] += bat_populations['U']
        total_bat_populations['R'] += bat_populations['R']
    
    total_res = {}
    table5_results_dict[speed]["Total"] = total_res
    
    total_res['fcc_count_total'] = total_fcc_counts['R'] + total_fcc_counts['U']
    total_res['fcc_count_U'] = total_fcc_counts['U']
    total_res['fcc_count_R'] = total_fcc_counts['R']

    total_res['bat_count_total'] = total_bat_counts['R'] + total_bat_counts['U']
    total_res['bat_count_U'] = total_bat_counts['U']
    total_res['bat_count_R'] = total_bat_counts['R']

    total_res['count_per_total'] = str(100 * total_res['bat_count_total'] / total_res['fcc_count_total'])
    total_res['count_per_U'] = str(100 * total_res['bat_count_U'] / total_res['fcc_count_U'])
    total_res['count_per_R'] = str(100 * total_res['bat_count_R'] / total_res['fcc_count_R'])

    total_res['fcc_pop_total'] = total_fcc_populations['R'] + total_fcc_populations['U']
    total_res['fcc_pop_U'] = total_fcc_populations['U']
    total_res['fcc_pop_R'] = total_fcc_populations['R']

    total_res['bat_pop_total'] = int(total_bat_populations['R'] + total_bat_populations['U'])
    total_res['bat_pop_U'] = int(total_bat_populations['U'])
    total_res['bat_pop_R'] = int(total_bat_populations['R'])

    total_res['pop_per_total'] = str(100 * total_res['bat_pop_total'] / total_res['fcc_pop_total'])
    total_res['pop_per_U'] = str(100 * total_res['bat_pop_U'] / total_res['fcc_pop_U'])
    total_res['pop_per_R'] = str(100 * total_res['bat_pop_R'] / total_res['fcc_pop_R'])

## Compute Table 11 (S1)

In [ ]:
# Table 11 (S1)
s1_states_dict = {0: {}, 25: {}}

for speed in [0, 25]:
    for state in ISPS_PER_STATE:
        
        state_dict = {}
        s1_states_dict[speed][state] = state_dict

        state_data = all_state_data[state]
        state_data_cols = data_cols[state]

        for address in state_data.itertuples():
            blockno = getattr(address, "addr_census_block")

            block_data = state_dict.get(blockno, [0, 0])

            local_col = state_data_cols["local"]
            local_speed_col = state_data_cols["local_speed"]
            local_speed_data = getattr(address, local_speed_col)

            has_local_ISP = False
            if getattr(address, local_col) == 1 and local_speed_data >= speed:
                block_data[0] += 1
                block_data[1] += 1
                has_local_ISP = True


            if not has_local_ISP:
                fcc_covered = False
                bat_covered = False
                bat_uncovered = False
                bad = False # unrecognized/unknown

                for isp in ISPS_PER_STATE[state]:
                    if bat_covered:
                        break

                    isp_fcc_covered = False

                    state_isp_data_cols = state_data_cols[isp]
                    fcc_cols = state_isp_data_cols["fcc"]
                    fcc_speed_cols = state_isp_data_cols["fcc_speed"]
                    tool_cols = state_isp_data_cols["tool"]
                    tool_speed_cols = state_isp_data_cols["tool_speed"]
                    timestamp_cols = state_isp_data_cols["timestamp"]

                    for i, fcc_col in enumerate(fcc_cols):
                        fcc_speed_col = fcc_speed_cols[i]
                        if getattr(address, fcc_col) == 1 and getattr(address, fcc_speed_col) >= speed:
                            isp_fcc_covered = True
                            fcc_covered = True

                    if not isp_fcc_covered:
                        continue

                    for i, tool_col in enumerate(tool_cols):
                        try:
                            tool_data = str(int(getattr(address, tool_col)))
                        except:
                            bad = True
                            continue

                        if tool_data in POS_RESPONSES[isp]: # covered by ISP (BAT)
                            bat_covered = True
                            break

                        elif tool_data in NEG_RESPONSES[isp]: # unrecognized/unknown/nan
                            bat_uncovered = True

                        elif tool_data not in UNRECOGNIZED_RESPONSES[isp]:
                            bad = True

                if bat_covered or (fcc_covered and bat_uncovered and not bad):
                    block_data[0] += 1 # FCC
                    if bat_covered:
                        block_data[1] += 1 # BAT

            state_dict[blockno] = block_data

In [ ]:
s1_states_results_dict = {0: {}, 25: {}}
for speed in [0, 25]:
    total_fcc_counts = {
        'U': 0,
        'R': 0
    }
    total_bat_counts = {
        'U': 0,
        'R': 0
    }
    total_fcc_populations = {
        'U': 0,
        'R': 0
    }
    total_bat_populations = {
        'U': 0,
        'R': 0
    }
    for state in s1_states_dict[speed]:
        print(state)

        res = {}
        s1_states_results_dict[speed][state] = res

        fcc_counts = {
            'U': 0,
            'R': 0
        }
        bat_counts = {
            'U': 0,
            'R': 0
        }
        fcc_populations = {
            'U': 0,
            'R': 0
        }
        bat_populations = {
            'U': 0,
            'R': 0
        }

        state_data = s1_states_dict[speed][state]

        for blockno in state_data:
            block_data = state_data[blockno]
            try:
                block_class = urban_rural_dict[blockno]
            except:
                continue

            if block_data[0] > 0:
                fcc_populations[block_class] += pop_dict[blockno]
                bat_populations[block_class] += pop_dict[blockno] * (block_data[1]/block_data[0])

                fcc_counts[block_class] += block_data[0]
                bat_counts[block_class] += block_data[1]

        res['fcc_count_total'] = fcc_counts['R'] + fcc_counts['U']
        res['fcc_count_U'] = fcc_counts['U']
        res['fcc_count_R'] = fcc_counts['R']

        res['bat_count_total'] = bat_counts['R'] + bat_counts['U']
        res['bat_count_U'] = bat_counts['U']
        res['bat_count_R'] = bat_counts['R']

        res['count_per_total'] = str(100 * res['bat_count_total'] / res['fcc_count_total'])
        res['count_per_U'] = str(100 * res['bat_count_U'] / res['fcc_count_U'])
        res['count_per_R'] = str(100 * res['bat_count_R'] / res['fcc_count_R'])

        res['fcc_pop_total'] = fcc_populations['R'] + fcc_populations['U']
        res['fcc_pop_U'] = fcc_populations['U']
        res['fcc_pop_R'] = fcc_populations['R']
        
        res['bat_pop_total'] = int(bat_populations['R'] + bat_populations['U'])
        res['bat_pop_U'] = int(bat_populations['U'])
        res['bat_pop_R'] = int(bat_populations['R'])
        
        res['pop_per_total'] = str(100 * res['bat_pop_total'] / res['fcc_pop_total'])
        res['pop_per_U'] = str(100 * res['bat_pop_U'] / res['fcc_pop_U'])
        res['pop_per_R'] = str(100 * res['bat_pop_R'] / res['fcc_pop_R'])
        
        total_fcc_counts['U'] += fcc_counts['U']
        total_fcc_counts['R'] += fcc_counts['R']
        total_bat_counts['U'] += bat_counts['U']
        total_bat_counts['R'] += bat_counts['R']
        
        total_fcc_populations['U'] += fcc_populations['U']
        total_fcc_populations['R'] += fcc_populations['R']
        total_bat_populations['U'] += bat_populations['U']
        total_bat_populations['R'] += bat_populations['R']
    
    total_res = {}
    s1_states_results_dict[speed]["Total"] = total_res
    
    total_res['fcc_count_total'] = total_fcc_counts['R'] + total_fcc_counts['U']
    total_res['fcc_count_U'] = total_fcc_counts['U']
    total_res['fcc_count_R'] = total_fcc_counts['R']

    total_res['bat_count_total'] = total_bat_counts['R'] + total_bat_counts['U']
    total_res['bat_count_U'] = total_bat_counts['U']
    total_res['bat_count_R'] = total_bat_counts['R']

    total_res['count_per_total'] = str(100 * total_res['bat_count_total'] / total_res['fcc_count_total'])
    total_res['count_per_U'] = str(100 * total_res['bat_count_U'] / total_res['fcc_count_U'])
    total_res['count_per_R'] = str(100 * total_res['bat_count_R'] / total_res['fcc_count_R'])

    total_res['fcc_pop_total'] = total_fcc_populations['R'] + total_fcc_populations['U']
    total_res['fcc_pop_U'] = total_fcc_populations['U']
    total_res['fcc_pop_R'] = total_fcc_populations['R']

    total_res['bat_pop_total'] = int(total_bat_populations['R'] + total_bat_populations['U'])
    total_res['bat_pop_U'] = int(total_bat_populations['U'])
    total_res['bat_pop_R'] = int(total_bat_populations['R'])

    total_res['pop_per_total'] = str(100 * total_res['bat_pop_total'] / total_res['fcc_pop_total'])
    total_res['pop_per_U'] = str(100 * total_res['bat_pop_U'] / total_res['fcc_pop_U'])
    total_res['pop_per_R'] = str(100 * total_res['bat_pop_R'] / total_res['fcc_pop_R'])

## Compute Table 12 (S2)

In [ ]:
# Table 12 (S2)
s2_states_dict = {0: {}, 25: {}}

for speed in [0, 25]:
    for state in ISPS_PER_STATE:

        state_dict = {}
        s2_states_dict[speed][state] = state_dict

        state_data = all_state_data[state]
        state_data_cols = data_cols[state]

        for address in state_data.itertuples():
            blockno = getattr(address, "addr_census_block")

            block_data = state_dict.get(blockno, [0, 0])

            local_col = state_data_cols["local"]
            local_speed_col = state_data_cols["local_speed"]
            local_speed_data = getattr(address, local_speed_col)

            has_local_ISP = False
            if getattr(address, local_col) == 1 and local_speed_data >= speed:
                block_data[0] += 1
                block_data[1] += 1
                has_local_ISP = True


            if not has_local_ISP:
                fcc_covered = False
                bat_covered = False
                bad = False

                for isp in ISPS_PER_STATE[state]:
                    if bat_covered:
                        break

                    isp_fcc_covered = False

                    state_isp_data_cols = state_data_cols[isp]
                    fcc_cols = state_isp_data_cols["fcc"]
                    fcc_speed_cols = state_isp_data_cols["fcc_speed"]
                    tool_cols = state_isp_data_cols["tool"]
                    tool_speed_cols = state_isp_data_cols["tool_speed"]
                    timestamp_cols = state_isp_data_cols["timestamp"]

                    for i, fcc_col in enumerate(fcc_cols):
                        fcc_speed_col = fcc_speed_cols[i]
                        if getattr(address, fcc_col) == 1 and getattr(address, fcc_speed_col) >= speed:
                            isp_fcc_covered = True
                            fcc_covered = True

                    if not isp_fcc_covered:
                        continue

                    for i, tool_col in enumerate(tool_cols):
                        try:
                            tool_data = str(int(getattr(address, tool_col)))
                        except:
                            bad = True
                            break

                        if tool_data in POS_RESPONSES[isp]: # covered by ISP (BAT)
                            bat_covered = True
                            break

                        elif tool_data not in NEG_RESPONSES[isp] + UNRECOGNIZED_RESPONSES[isp] + BUSINESS_RESPONSES[isp] + UNKNOWN_RESPONSES[isp]:
                            bad = True

                if bat_covered or (fcc_covered and not bad):
                    block_data[0] += 1 # FCC
                    if bat_covered:
                        block_data[1] += 1 # BAT

            state_dict[blockno] = block_data

In [ ]:
s2_states_results_dict = {0: {}, 25: {}}
for speed in [0, 25]:
    total_fcc_counts = {
        'U': 0,
        'R': 0
    }
    total_bat_counts = {
        'U': 0,
        'R': 0
    }
    total_fcc_populations = {
        'U': 0,
        'R': 0
    }
    total_bat_populations = {
        'U': 0,
        'R': 0
    }
    for state in s2_states_dict[speed]:
        print(state)

        res = {}
        s2_states_results_dict[speed][state] = res

        fcc_counts = {
            'U': 0,
            'R': 0
        }
        bat_counts = {
            'U': 0,
            'R': 0
        }
        fcc_populations = {
            'U': 0,
            'R': 0
        }
        bat_populations = {
            'U': 0,
            'R': 0
        }

        state_data = s2_states_dict[speed][state]

        for blockno in state_data:
            block_data = state_data[blockno]
            try:
                block_class = urban_rural_dict[blockno]
            except:
                continue

            if block_data[0] > 0:
                fcc_populations[block_class] += pop_dict[blockno]
                bat_populations[block_class] += pop_dict[blockno] * (block_data[1]/block_data[0])

                fcc_counts[block_class] += block_data[0]
                bat_counts[block_class] += block_data[1]

        res['fcc_count_total'] = fcc_counts['R'] + fcc_counts['U']
        res['fcc_count_U'] = fcc_counts['U']
        res['fcc_count_R'] = fcc_counts['R']

        res['bat_count_total'] = bat_counts['R'] + bat_counts['U']
        res['bat_count_U'] = bat_counts['U']
        res['bat_count_R'] = bat_counts['R']

        res['count_per_total'] = str(100 * res['bat_count_total'] / res['fcc_count_total'])
        res['count_per_U'] = str(100 * res['bat_count_U'] / res['fcc_count_U'])
        res['count_per_R'] = str(100 * res['bat_count_R'] / res['fcc_count_R'])

        res['fcc_pop_total'] = fcc_populations['R'] + fcc_populations['U']
        res['fcc_pop_U'] = fcc_populations['U']
        res['fcc_pop_R'] = fcc_populations['R']
        
        res['bat_pop_total'] = int(bat_populations['R'] + bat_populations['U'])
        res['bat_pop_U'] = int(bat_populations['U'])
        res['bat_pop_R'] = int(bat_populations['R'])
        
        res['pop_per_total'] = str(100 * res['bat_pop_total'] / res['fcc_pop_total'])
        res['pop_per_U'] = str(100 * res['bat_pop_U'] / res['fcc_pop_U'])
        res['pop_per_R'] = str(100 * res['bat_pop_R'] / res['fcc_pop_R'])
    
        total_fcc_counts['U'] += fcc_counts['U']
        total_fcc_counts['R'] += fcc_counts['R']
        total_bat_counts['U'] += bat_counts['U']
        total_bat_counts['R'] += bat_counts['R']
        
        total_fcc_populations['U'] += fcc_populations['U']
        total_fcc_populations['R'] += fcc_populations['R']
        total_bat_populations['U'] += bat_populations['U']
        total_bat_populations['R'] += bat_populations['R']
    
    total_res = {}
    s2_states_results_dict[speed]["Total"] = total_res
    
    total_res['fcc_count_total'] = total_fcc_counts['R'] + total_fcc_counts['U']
    total_res['fcc_count_U'] = total_fcc_counts['U']
    total_res['fcc_count_R'] = total_fcc_counts['R']

    total_res['bat_count_total'] = total_bat_counts['R'] + total_bat_counts['U']
    total_res['bat_count_U'] = total_bat_counts['U']
    total_res['bat_count_R'] = total_bat_counts['R']

    total_res['count_per_total'] = str(100 * total_res['bat_count_total'] / total_res['fcc_count_total'])
    total_res['count_per_U'] = str(100 * total_res['bat_count_U'] / total_res['fcc_count_U'])
    total_res['count_per_R'] = str(100 * total_res['bat_count_R'] / total_res['fcc_count_R'])

    total_res['fcc_pop_total'] = total_fcc_populations['R'] + total_fcc_populations['U']
    total_res['fcc_pop_U'] = total_fcc_populations['U']
    total_res['fcc_pop_R'] = total_fcc_populations['R']

    total_res['bat_pop_total'] = int(total_bat_populations['R'] + total_bat_populations['U'])
    total_res['bat_pop_U'] = int(total_bat_populations['U'])
    total_res['bat_pop_R'] = int(total_bat_populations['R'])

    total_res['pop_per_total'] = str(100 * total_res['bat_pop_total'] / total_res['fcc_pop_total'])
    total_res['pop_per_U'] = str(100 * total_res['bat_pop_U'] / total_res['fcc_pop_U'])
    total_res['pop_per_R'] = str(100 * total_res['bat_pop_R'] / total_res['fcc_pop_R'])

## Compute Table 13 (S3)

In [ ]:
# Table 13 (S3)
s3_states_dict = {0: {}, 25: {}}

for speed in [0, 25]:
    for state in ISPS_PER_STATE:
        state_dict = {}
        s3_states_dict[speed][state] = state_dict

        state_data = all_state_data[state]
        state_data_cols = data_cols[state]

        for address in state_data.itertuples():
            blockno = getattr(address, "addr_census_block")

            block_data = state_dict.get(blockno, [0, 0])

            local_col = state_data_cols["local"]
            local_speed_col = state_data_cols["local_speed"]
            local_speed_data = getattr(address, local_speed_col)

            fcc_covered = False
            bat_covered = False
            bad = False

            for isp in ISPS_PER_STATE[state]:
                if bat_covered:
                    break
                    
                state_isp_data_cols = state_data_cols[isp]
                fcc_cols = state_isp_data_cols["fcc"]
                fcc_speed_cols = state_isp_data_cols["fcc_speed"]
                tool_cols = state_isp_data_cols["tool"]
                tool_speed_cols = state_isp_data_cols["tool_speed"]
                timestamp_cols = state_isp_data_cols["timestamp"]

                isp_fcc_covered = False

                for i, fcc_col in enumerate(fcc_cols):
                    fcc_speed_col = fcc_speed_cols[i]
                    if getattr(address, fcc_col) == 1 and getattr(address, fcc_speed_col) >= speed:
                        isp_fcc_covered = True
                        fcc_covered = True
                        break

                if not isp_fcc_covered:
                    continue

                for i, tool_col in enumerate(tool_cols):
                    timestamp_col = timestamp_cols[i]
                    timestamp = getattr(address, timestamp_col) 

                    if timestamp > PAPER_TIME:
                        bad = True
                        continue

                    try:
                        tool_data = str(int(getattr(address, tool_col)))
                    except:
                        bad = True
                        break

                    if tool_data in POS_RESPONSES[isp]: # covered by ISP (BAT)
                        bat_covered = True
                        break

                    elif tool_data not in NEG_RESPONSES[isp]:
                        bad = True

            if bat_covered or (fcc_covered and not bad):
                block_data[0] += 1 # FCC
                if has_local_ISP or bat_covered:
                    block_data[1] += 1 # BAT

            state_dict[blockno] = block_data

In [ ]:
s3_states_results_dict = {0: {}, 25: {}}
for speed in [0, 25]:
    total_fcc_counts = {
        'U': 0,
        'R': 0
    }
    total_bat_counts = {
        'U': 0,
        'R': 0
    }
    total_fcc_populations = {
        'U': 0,
        'R': 0
    }
    total_bat_populations = {
        'U': 0,
        'R': 0
    }
    for state in s3_states_dict[speed]:
        print(state)

        res = {}
        s3_states_results_dict[speed][state] = res

        fcc_counts = {
            'U': 0,
            'R': 0
        }
        bat_counts = {
            'U': 0,
            'R': 0
        }
        fcc_populations = {
            'U': 0,
            'R': 0
        }
        bat_populations = {
            'U': 0,
            'R': 0
        }

        state_data = s3_states_dict[speed][state]

        for blockno in state_data:
            block_data = state_data[blockno]
            try:
                block_class = urban_rural_dict[blockno]
            except:
                continue

            if block_data[0] > 0:
                fcc_populations[block_class] += pop_dict[blockno]
                bat_populations[block_class] += pop_dict[blockno] * (block_data[1]/block_data[0])
                
                fcc_counts[block_class] += block_data[0]
                bat_counts[block_class] += block_data[1]

        res['fcc_count_total'] = fcc_counts['R'] + fcc_counts['U']
        res['fcc_count_U'] = fcc_counts['U']
        res['fcc_count_R'] = fcc_counts['R']

        res['bat_count_total'] = bat_counts['R'] + bat_counts['U']
        res['bat_count_U'] = bat_counts['U']
        res['bat_count_R'] = bat_counts['R']

        res['count_per_total'] = str(100 * res['bat_count_total'] / res['fcc_count_total'])
        res['count_per_U'] = str(100 * res['bat_count_U'] / res['fcc_count_U'])
        res['count_per_R'] = str(100 * res['bat_count_R'] / res['fcc_count_R'])

        res['fcc_pop_total'] = fcc_populations['R'] + fcc_populations['U']
        res['fcc_pop_U'] = fcc_populations['U']
        res['fcc_pop_R'] = fcc_populations['R']
        
        res['bat_pop_total'] = int(bat_populations['R'] + bat_populations['U'])
        res['bat_pop_U'] = int(bat_populations['U'])
        res['bat_pop_R'] = int(bat_populations['R'])
        
        res['pop_per_total'] = str(100 * res['bat_pop_total'] / res['fcc_pop_total'])
        res['pop_per_U'] = str(100 * res['bat_pop_U'] / res['fcc_pop_U'])
        res['pop_per_R'] = str(100 * res['bat_pop_R'] / res['fcc_pop_R'])
        
        total_fcc_counts['U'] += fcc_counts['U']
        total_fcc_counts['R'] += fcc_counts['R']
        total_bat_counts['U'] += bat_counts['U']
        total_bat_counts['R'] += bat_counts['R']
        
        total_fcc_populations['U'] += fcc_populations['U']
        total_fcc_populations['R'] += fcc_populations['R']
        total_bat_populations['U'] += bat_populations['U']
        total_bat_populations['R'] += bat_populations['R']
    
    total_res = {}
    s3_states_results_dict[speed]["Total"] = total_res
    
    total_res['fcc_count_total'] = total_fcc_counts['R'] + total_fcc_counts['U']
    total_res['fcc_count_U'] = total_fcc_counts['U']
    total_res['fcc_count_R'] = total_fcc_counts['R']

    total_res['bat_count_total'] = total_bat_counts['R'] + total_bat_counts['U']
    total_res['bat_count_U'] = total_bat_counts['U']
    total_res['bat_count_R'] = total_bat_counts['R']

    total_res['count_per_total'] = str(100 * total_res['bat_count_total'] / total_res['fcc_count_total'])
    total_res['count_per_U'] = str(100 * total_res['bat_count_U'] / total_res['fcc_count_U'])
    total_res['count_per_R'] = str(100 * total_res['bat_count_R'] / total_res['fcc_count_R'])

    total_res['fcc_pop_total'] = total_fcc_populations['R'] + total_fcc_populations['U']
    total_res['fcc_pop_U'] = total_fcc_populations['U']
    total_res['fcc_pop_R'] = total_fcc_populations['R']

    total_res['bat_pop_total'] = int(total_bat_populations['R'] + total_bat_populations['U'])
    total_res['bat_pop_U'] = int(total_bat_populations['U'])
    total_res['bat_pop_R'] = int(total_bat_populations['R'])

    total_res['pop_per_total'] = str(100 * total_res['bat_pop_total'] / total_res['fcc_pop_total'])
    total_res['pop_per_U'] = str(100 * total_res['bat_pop_U'] / total_res['fcc_pop_U'])
    total_res['pop_per_R'] = str(100 * total_res['bat_pop_R'] / total_res['fcc_pop_R'])

## Print Tables as LaTeX

In [ ]:
for state in STATES + ["Total"]:
    speed_0 = table5_dict[0][state] # SET TABLE (table5_dict, s1_states_results_dict, s2_states_results_dict, s3_states_results_dict)
    speed_25 = table5_dict[25][state] # SET TABLE (table5_dict, s1_states_results_dict, s2_states_results_dict, s3_states_results_dict)
    
    line1 = [
        speed_0['fcc_count_total'], speed_0['bat_count_total'], speed_0['count_per_total'] , r"\%",
        speed_25['fcc_count_total'], speed_25['bat_count_total'], speed_25['count_per_total'] , r"\%",
        speed_0['fcc_pop_total'], speed_0['bat_pop_total'], speed_0['pop_per_total'] , r"\%",
        speed_25['fcc_pop_total'], speed_25['bat_pop_total'], speed_25['pop_per_total'] , r"\%"
    ]
    line2 = [
        speed_0['fcc_count_U'], speed_0['bat_count_U'], speed_0['count_per_U'] , r"\%",
        speed_25['fcc_count_U'], speed_25['bat_count_U'], speed_25['count_per_U'] , r"\%",
        speed_0['fcc_pop_U'], speed_0['bat_pop_U'], speed_0['pop_per_U'] , r"\%",
        speed_25['fcc_pop_U'], speed_25['bat_pop_U'], speed_25['pop_per_U'] , r"\%"
    ]
    line3 = [
        speed_0['fcc_count_R'], speed_0['bat_count_R'], speed_0['count_per_R'] , r"\%",
        speed_25['fcc_count_R'], speed_25['bat_count_R'], speed_25['count_per_R'] , r"\%",
        speed_0['fcc_pop_R'], speed_0['bat_pop_R'], speed_0['pop_per_R'] , r"\%",
        speed_25['fcc_pop_R'], speed_25['bat_pop_R'], speed_25['pop_per_R'] , r"\%"
    ]
    
    line1_str = r"\rowcolor[HTML]{C0C0C0}" + LABELS[state]
    line1_str += " & All & " + " & ".join(["{:,.2f}".format(float(x)) if i % 4 != 3 else x for i, x in enumerate(line1)]).replace(".00", "").replace(r" & \%", r"\%") + r" \\"
    
    line2_str = " & Urban & " + " & ".join(["{:,.2f}".format(float(x)) if i % 4 != 3 else x for i, x in enumerate(line2)]).replace(".00", "").replace(r" & \%", r"\%") + r" \\"
    
    line3_str = " & Rural & " + " & ".join(["{:,.2f}".format(float(x)) if i % 4 != 3 else x for i, x in enumerate(line3)]).replace(".00", "").replace(r" & \%", r"\%") + r" \\"
    
    print(line1_str)
    print(line2_str)
    print(line3_str)

# Figures/Misc Tables

## Figure 3: CDF

In [ ]:
all_isp_data = defaultdict(lambda: defaultdict(lambda: Counter()))

for state in all_state_data:
    print(state)
    state_data = all_state_data[state]

    for isp in ISPS_PER_STATE[state]:
        state_isp_data_cols = data_cols[state][isp]
        
        fcc_cols = state_isp_data_cols["fcc"]
        tool_cols = state_isp_data_cols["tool"]
        
        for address in state_data.itertuples():
            blockno = address.addr_census_block
            
            fcc_covered = False
            for fcc_col in fcc_cols:
                if getattr(address, fcc_col) == 1:
                    all_isp_data[isp][blockno]['fcc'] += 1
                    fcc_covered = True
                    break
                    
            if not fcc_covered:
                continue
            
            bat_covered = False
            bad = False
                
            for tool_col in tool_cols:
                try:
                    tool_data = str(int(getattr(address, tool_col)))
                except:
                    continue
                if tool_data in POS_RESPONSES[isp]:
                    bat_covered = True
                    all_isp_data[isp][blockno]['tool_pos'] += 1
                    break
                elif tool_data not in NEG_RESPONSES[isp]:
                    bad = True
            
            if bat_covered or not bad:
                all_isp_data[isp][blockno]['tool_neg'] += 1

In [ ]:
fig, axes = plt.subplots()
handles = []

for isp, per_isp_data in sorted(all_isp_data.items(), key=lambda x: SORTED_ISPS.index(x[0])):
    ratios = []
    
    for block, per_isp_block_data in per_isp_data.items():
        fcc = per_isp_block_data["fcc"]
        pos = per_isp_block_data["tool_pos"]
        neg = per_isp_block_data["tool_neg"]
        if pos+neg > 0:
            ratios.append(pos / neg)

    sorted_ratios = sorted(ratios)
                      
    x_axis = np.arange(1, len(sorted_ratios)+1) / len(sorted_ratios)
    
    line, = axes.step(x_axis, sorted_ratios, label=LABELS[isp])
#     handles.append(line)
    
plt.legend()
plt.ylabel("Percentage of Block with Coverage")
plt.xlabel("Proportion of Blocks")
axes.grid(False)
vals = axes.get_yticks()
axes.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
fig.tight_layout()
plt.savefig("coverage_distribution.pdf")

## Table 3

In [ ]:
tab3_data_0 = defaultdict(Counter)
tab3_data_25 = defaultdict(Counter)

for speed in [0, 25]:
    for state, state_data in all_state_data.items():
        for isp in ISPS_PER_STATE[state]:
            state_isp_data_cols = data_cols[state][isp]
            fcc_cols = state_isp_data_cols["fcc"]
            fcc_speed_cols = state_isp_data_cols["fcc_speed"]
            tool_cols = state_isp_data_cols["tool"]
            tool_speed_cols = state_isp_data_cols["tool_speed"]

            isp_data = state_data[fcc_cols + fcc_speed_cols + tool_cols]
            isp_data = isp_data[(isp_data[fcc_cols] == 1).any(axis=1)]
            isp_data = isp_data[(isp_data[fcc_speed_cols] >= speed).any(axis=1)]
            
            cov_data = isp_data[(isp_data[tool_cols].isin(POS_RESPONSES[isp]).any(axis=1))]
            
            other_data = isp_data[~(isp_data[tool_cols].isin(POS_RESPONSES[isp]).any(axis=1))]
            
            noncov_data = other_data[(other_data[tool_cols].isin(NEG_RESPONSES[isp]).all(axis=1))]
            
            if speed == 0:
                tab3_data_0[isp]['fcc'] += len(cov_data) + len(noncov_data)
                tab3_data_0[isp]['bat'] += len(cov_data)
            if speed == 25:
                tab3_data_25[isp]['fcc'] += len(cov_data) + len(noncov_data)
                tab3_data_25[isp]['bat'] += len(cov_data)

In [ ]:
tab3_data_0

In [ ]:
tab3_data_25

## Table 4

In [ ]:
block_count_0 = Counter()
block_count_25 = Counter()

overreporting_counts_0 = Counter()
overreporting_counts_25 = Counter()

def overcount_block(block_data, isp, fcc_cols, fcc_speed_cols, tool_cols, timestamp_cols):
    if len(block_data) > 0:
            block_count_0[isp] += 1
    for speed in [0, 25]:
        if speed == 25:
            block_data = block_data[(block_data[fcc_speed_cols] >= speed).any(axis=1)]
            if len(block_data) > 0:
                block_count_25[isp] += 1
                
        if len(block_data) <= 20:
            return

        if len(block_data) > 0 and len(block_data) == block_data[tool_cols].isin(NEG_RESPONSES[isp]).all(axis=1).sum():
            if speed == 0:
                overreporting_counts_0[isp] += 1
            else:
                overreporting_counts_25[isp] += 1

for state, state_data in all_state_data.items():
    print(state)
    for isp in ISPS_PER_STATE[state]:
        
        isp_cols = data_cols[state][isp]
        fcc_cols = isp_cols["fcc"]
        fcc_speed_cols = isp_cols["fcc_speed"]
        tool_cols = isp_cols["tool"]
        
        state_data[(state_data[fcc_cols] == 1).any(axis=1)].groupby("addr_census_block").apply(lambda df: overcount_block(df, isp, fcc_cols, fcc_speed_cols, tool_cols, timestamp_cols))

In [ ]:
overreporting_counts_0

In [ ]:
overreporting_counts_25

In [ ]:
block_count_0

In [ ]:
block_count_25

## Figure 5

#### Vectorized

In [ ]:
speed_isps = ['att', 'centurylink', 'consolidated', 'windstream']
all_isp_speed_max = defaultdict(list)

for state in all_state_data:
    state_data_global = all_state_data[state]
    
    isps = [x for x in speed_isps if x in ISPS_PER_STATE[state]]
    
    for isp in isps:
        state_isp_data_cols = data_cols[state][isp]
        fcc_cols = state_isp_data_cols["fcc"]
        fcc_speed_cols = state_isp_data_cols["fcc_speed"]
        tool_cols = state_isp_data_cols["tool"]
        tool_speed_cols = state_isp_data_cols["tool_speed"]
        
        state_data.loc[:, "FCC"] = state_data[fcc_speed_cols].max(axis=1)
        state_data.loc[:, "BATs"] = state_data[tool_speed_cols].max(axis=1)

        all_isp_speed_max[isp].append(state_data[["addr_id", "addr_census_block", "FCC", "BATs"]])
        
for isp, isp_speed_data in list(all_isp_speed_max.items()):
    all_isp_speed_max[isp] = pd.concat(isp_speed_data).dropna()

In [ ]:
fig, axes = plt.subplots(ncols=4, sharey=True, figsize=(16, 6))
speed_isps = ['att', 'centurylink', 'consolidated', 'windstream']
for i, (isp, isp_speed_data) in enumerate(sorted(all_isp_speed_max.items(), key=lambda x: speed_isps.index(x[0]))):
    isp_speed_data = isp_speed_data.copy(deep=True)

    isp_speed_data = isp_speed_data[isp_speed_data["BATs"] > 0]
    
    isp_speed_data["urban/rural"] = isp_speed_data["addr_census_block"].transform(lambda x: LABELS[urban_rural_dict[x]])
    new_copy = isp_speed_data.copy(deep=True)
    new_copy["urban/rural"] = "Total"
    
    isp_speed_data = pd.concat([isp_speed_data, new_copy])
    
    melted = isp_speed_data.melt(id_vars=['addr_census_block', 'urban/rural', 'addr_id'], value_vars=['FCC', 'BATs'], var_name='source', value_name='max_speed')
    
    ax = sns.boxplot(x='urban/rural', y='max_speed', hue='source', data=melted, showfliers=False, ax=axes[i], order=['Total', 'Urban', 'Rural'], palette="Blues")
    ax.set_yscale('log')
    ax.set_xlabel("")
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels)
    if i == 0:
        ax.set_ylabel("Maximum Download Speed (Mbps)")
    else:
        ax.set_ylabel("")
        ax.get_legend().remove()
    ax.set_title("{}".format(LABELS[isp]))

fig.savefig("fig5.pdf")

#### Iterative

In [ ]:
speed_isps = ['att', 'centurylink', 'consolidated', 'windstream']
all_isp_speed_max_list = []

for state in all_state_data:
    print(state)
    state_data_global = all_state_data[state]
    
    isps = [x for x in speed_isps if x in ISPS_PER_STATE[state]]
    
    for address in state_data_global.itertuples():
        blockno = getattr(address, "addr_census_block")  
        addr_id = getattr(address, "addr_id")
        try:
            block_class = urban_rural_dict[blockno]
        except:
            continue
        
        for isp in isps:
            isp_label = LABELS[isp]
            state_isp_data_cols = data_cols[state][isp]
            fcc_cols = state_isp_data_cols["fcc"]
            fcc_speed_cols = state_isp_data_cols["fcc_speed"]
            tool_cols = state_isp_data_cols["tool"]
            tool_speed_cols = state_isp_data_cols["tool_speed"]
            
            fcc_max = float('-inf')
            tool_max = float('-inf')
            for i, tool_col in enumerate(tool_cols):
                fcc_col = fcc_cols[i]
                fcc_speed_col = fcc_speed_cols[i]
                tool_speed_col = tool_speed_cols[i]
                
                if getattr(address, fcc_col) == 1:
                    fcc_max = max(fcc_max, getattr(address, fcc_speed_col))
                    if str(int(getattr(address, tool_col))) in POS_RESPONSES[isp]:
                        tool_max = max(tool_max, getattr(address, tool_speed_col))
                
            if tool_max > 0:
                all_isp_speed_max_list.append([addr_id, blockno, LABELS[block_class], isp, "FCC", fcc_max])
                all_isp_speed_max_list.append([addr_id, blockno, LABELS[block_class], isp, "BATs", tool_max])

                all_isp_speed_max_list.append([addr_id, blockno, "Total", isp, "FCC", fcc_max])
                all_isp_speed_max_list.append([addr_id, blockno, "Total", isp, "BATs", tool_max])

column_names = ["addr_id", "blockno", "Urban/Rural", "ISP", "Source", "Maximum Download Speed"]
isp_speed_data_iter = pd.DataFrame(all_isp_speed_max_list, columns=column_names)

In [ ]:
fig, axes = plt.subplots(ncols=4, sharey=True, figsize=(16, 6))

for i, isp in enumerate(speed_isps):
    print(isp)
    isp_data = isp_speed_data_iter.query(f"ISP == '{isp}'")

    ax = sns.boxplot(x='Urban/Rural', y='Maximum Download Speed', hue='Source', data=isp_data, showfliers=False, ax=axes[i], order=['Total', 'Urban', 'Rural'], palette="Blues")
    ax.set_yscale('log')
    ax.set_xlabel("")
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels)
    if i == 0:
        ax.set_ylabel("Maximum Download Speed (Mbps)")
    else:
        ax.set_ylabel("")
        ax.get_legend().remove()
    ax.set_title("{}".format(LABELS[isp]))

fig.tight_layout()
fig.savefig("fig5.pdf")

## Table 10

In [ ]:
isp_stats = defaultdict(Counter)

for state, state_data in all_state_data.items():
    print(state)
    for isp in ISPS_PER_STATE[state]:
        isp_cols = data_cols[state][isp]
        tool_cols = isp_cols["tool"]
        
        pos = False
        neg = False
        unrecognized = False
        business = False
        unknown = False

        isp_stats[isp]["pos"] += (state_data[tool_cols].isin(POS_RESPONSES[isp]).any(axis=1)).sum()
        isp_stats[isp]["neg"] += (state_data[tool_cols].isin(NEG_RESPONSES[isp]).all(axis=1)).sum()
        isp_stats[isp]["unrecognized"] += (state_data[tool_cols].isin(UNRECOGNIZED_RESPONSES[isp]).all(axis=1)).sum()
        isp_stats[isp]["business"] += (state_data[tool_cols].isin(BUSINESS_RESPONSES[isp]).all(axis=1)).sum()
        isp_stats[isp]["unknown"] += (state_data[tool_cols].isin(UNKNOWN_RESPONSES[isp]).all(axis=1)).sum()
isp_stats

## Precompute for Fig 6 + Fig 9 + Regression

In [ ]:
isps_per_block = defaultdict(set) # Regression
isps_per_block_cov_noncov_only_0 = defaultdict(lambda: defaultdict(list)) # Table 6
isps_per_block_cov_noncov_only_25 = defaultdict(lambda: defaultdict(list)) # Table 6

for state, state_data in all_state_data.items():
    state_data_cols = data_cols[state]
    for address in state_data.itertuples():
        blockno = getattr(address, "addr_census_block")
        bad = False
        
        fcc_isps_0 = []
        fcc_isps_25 = []
        bat_isps = []
        
        for isp in ISPS_PER_STATE[state]:
            state_isp_data_cols = state_data_cols[isp]
            fcc_cols = state_isp_data_cols["fcc"]
            fcc_speed_cols = state_isp_data_cols["fcc_speed"]
            tool_cols = state_isp_data_cols["tool"]
            tool_speed_cols = state_isp_data_cols["tool_speed"]
            
            fcc_covered = False
            fcc_covered_25 = False
            for i, fcc_col in enumerate(fcc_cols):
                fcc_speed_col = fcc_speed_cols[i]
                if getattr(address, fcc_col) == 1:
                    fcc_covered = True
                    if getattr(address, fcc_speed_col) >= 25:
                        fcc_covered_25 = True

            if not fcc_covered:
                continue
                
            fcc_isps_0.append(isp)
            if fcc_covered_25:
                fcc_isps_25.append(isp)
            
            bat_covered = False

            for i, tool_col in enumerate(tool_cols):
                tool_speed_col = tool_speed_cols[i]

                tool_data = getattr(address, tool_col)
                if pd.isna(tool_data):
                    bad = True
                    break
                
                tool_data = str(int(tool_data))
                
                if tool_data in POS_RESPONSES[isp]:
                    bat_covered = True
                elif tool_data not in NEG_RESPONSES[isp]:
                    bad = True
                    break

            if bat_covered:
                bat_isps.append(isp)
                
        for isp in fcc_isps_0:
            isps_per_block[blockno].add(isp)
            
        if not bad and len(fcc_isps_0) > 0:
            isps_per_block_cov_noncov_only_0[blockno]['fcc'].append(len(fcc_isps_0))
            isps_per_block_cov_noncov_only_0[blockno]['tool'].append(len(bat_isps))
            
            if len(fcc_isps_25) > 0:
                isps_per_block_cov_noncov_only_25[blockno]['fcc'].append(len(fcc_isps_0))
                isps_per_block_cov_noncov_only_25[blockno]['tool'].append(len(bat_isps))

## Table 6 (Regression)

In [ ]:
ACS_race = pd.read_csv("ACS/ACSDT5Y2018.B03002_data_with_overlays_2020-12-21T003055.csv")[1:][["GEO_ID", "B03002_003E", "B03002_004E", "B03002_005E", "B03002_006E", "B03002_007E", "B03002_008E", "B03002_009E", "B03002_012E"]]
ACS_poverty = pd.read_csv("ACS/ACSST5Y2018.S1701_data_with_overlays_2020-12-21T002937.csv")[1:][["GEO_ID", "S1701_C03_001E"]]

ACS_race["GEO_ID"] = ACS_race["GEO_ID"].transform(lambda x: x.split("US")[1]).astype(int)
for col in ACS_race:
    if col != "GEO_ID":
        ACS_race[col] = ACS_race[col].astype(int)

ACS_poverty["GEO_ID"] = ACS_poverty["GEO_ID"].transform(lambda x: x.split("US")[1]).astype(int)

ACS_poverty = ACS_poverty[ACS_poverty["S1701_C03_001E"] != '-']
ACS_poverty["S1701_C03_001E"] = ACS_poverty["S1701_C03_001E"].astype(float)
ACS_poverty["S1701_C03_001E"] = ACS_poverty["S1701_C03_001E"].transform(lambda x: x/100.0)
ACS_poverty = ACS_poverty.rename(columns={'S1701_C03_001E': 'pov_rate'})

In [ ]:
ACS_race_dict = ACS_race.set_index("GEO_ID").to_dict('index')
ACS_poverty_dict = ACS_poverty.set_index("GEO_ID").to_dict('index')

In [ ]:
minority_count_per_tract = defaultdict(dict)

for tractno, race_data in ACS_race_dict.items():
    white = race_data["B03002_003E"]
    minority = sum(race_data.values()) - white
    
    minority_count_per_tract[tractno]['white'] = white
    minority_count_per_tract[tractno]['minority'] = minority
    
poverty_percent_per_tract = {}

for tractno, poverty_rate in ACS_poverty_dict.items():
    poverty_percent_per_tract[tractno] = poverty_rate['pov_rate']

In [ ]:
covered_counts_per_tract = Counter()
not_covered_counts_per_tract = Counter()
rural_urban_total_counts_per_tract = defaultdict(lambda:{
    'U': 0,
    'R': 0,
})

for state, state_data in table5_dict[0].items():
    for blockno in state_data:
        if blockno not in urban_rural_dict:
            continue
        block_class = urban_rural_dict[blockno]
        if state_data[blockno][0] > 0:
            tractno = int(blockno / 10000)
            covered_counts_per_tract[tractno] += state_data[blockno][1]
            not_covered_counts_per_tract[tractno] += state_data[blockno][0]
            
            rural_urban_total_counts_per_tract[tractno][block_class] += state_data[blockno][0]

isps_per_tract = defaultdict(Counter)
blocks_per_tract = Counter()
for blockno, isps in isps_per_block.items():
    if len(isps) == 0:
        continue
    tractno = int(blockno / 10000)
    blocks_per_tract[tractno] += 1
    
    for isp in isps:
        isps_per_tract[tractno][isp] += 1

for tractno in isps_per_tract:
    for isp in isps_per_tract[tractno]:
        isps_per_tract[tractno][isp] = isps_per_tract[tractno][isp] / blocks_per_tract[tractno]
        
pop_per_tract = Counter()

for blockno, pop in pop_dict.items():
    tractno = int(blockno / 10000)
    pop_per_tract[tractno] += pop

In [ ]:
column_names=['tract', 'coverage_avg','minority_per','poverty_per','rural_per','state', 'pop'] + SORTED_ISPS

In [ ]:
data_for_df = []

arr = []

excluded_tracts = []

for tractno in rural_urban_total_counts_per_tract:
    state_fips = int(tractno / 10**9)
    if state_fips not in VALID_STATE_FIPS:
        excluded_tracts.append(state_fips)
        continue
    state = VALID_STATE_FIPS[state_fips]
    arr.append((tractno, tractno not in minority_count_per_tract, tractno not in poverty_percent_per_tract, tractno not in covered_counts_per_tract, tractno not in not_covered_counts_per_tract))
    if tractno not in minority_count_per_tract or tractno not in poverty_percent_per_tract or (tractno not in covered_counts_per_tract and tractno not in not_covered_counts_per_tract):
        continue
    data_for_df.append([
        tractno,
        covered_counts_per_tract[tractno]/not_covered_counts_per_tract[tractno],
        minority_count_per_tract[tractno]['minority']/(minority_count_per_tract[tractno]['minority']+minority_count_per_tract[tractno]['white']),
        poverty_percent_per_tract[tractno],
        rural_urban_total_counts_per_tract[tractno]['R']/(rural_urban_total_counts_per_tract[tractno]['R']+rural_urban_total_counts_per_tract[tractno]['U']),
        state,
        pop_per_tract[tractno],
        isps_per_tract[tractno]['att'],
        isps_per_tract[tractno]['centurylink'],
        isps_per_tract[tractno]['consolidated'],
        isps_per_tract[tractno]['cox'],
        isps_per_tract[tractno]['xfinity'],
        isps_per_tract[tractno]['verizon'],
        isps_per_tract[tractno]['windstream'],
        isps_per_tract[tractno]['charter'],
        isps_per_tract[tractno]['frontier'],
    ])
    
df_tract_data = pd.DataFrame(data_for_df, columns=column_names)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

mod = smf.ols(formula='coverage_avg ~ minority_per + poverty_per + rural_per + C(state) + pop + att + centurylink + consolidated + cox + xfinity + verizon + windstream + charter + frontier', data=df_tract_data)
lm = mod.fit()
print(lm.summary())

## Figure 6 + 9 (Competition by Urban/Rural and Speed)

### Figure 6 (Competition by Urban/Rural)

In [ ]:
competition_state_stats = defaultdict(list)
column_names = ['state', 'block', 'Rural/Urban', 'avg_tool_count', 'avg_fcc_count', 'overstatement']

for blockno, block_data in isps_per_block_cov_noncov_only_0.items():
    if blockno not in urban_rural_dict:
        continue
    block_class = urban_rural_dict[blockno]
    state_fips = int(blockno / 10**13)
    state = VALID_STATE_FIPS[state_fips]
    
    fcc_data = block_data["fcc"]
    tool_data = block_data["tool"]
    fcc_count_avg = sum(fcc_data)/len(fcc_data)
    tool_count_avg = sum(tool_data)/len(tool_data)
    competition_state_stats[state].append([state, blockno, LABELS[block_class], tool_count_avg, fcc_count_avg, tool_count_avg / fcc_count_avg])
    competition_state_stats[state].append([state, blockno, "Total", tool_count_avg, fcc_count_avg, tool_count_avg / fcc_count_avg])

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(10,10), sharey=True)

for i, state in enumerate(STATES):
    ax_row = int(i / (len(STATES)/3))
    ax_col = int(i % (len(STATES)/3))
    df_counts_per_block = pd.DataFrame(competition_state_stats[state], columns = column_names)
    
    ax = sns.boxplot(x = 'Rural/Urban', y = 'overstatement', data = df_counts_per_block,fliersize=0,ax=axes[ax_row][ax_col], order=['Total','Urban','Rural']) 
    ax.set_ylabel("")
        #ax.get_legend().remove()
    ax.set_xlabel("")
    ax.set_title("{}".format(LABELS[state]))
    ax.grid(False)

fig.tight_layout()
fig.savefig("fig6.pdf")

### Figure 9 (Competition by Speed)

In [ ]:
competition_state_stats = defaultdict(list)
column_names = ['state', 'block', 'speed', 'avg_tool_count', 'avg_fcc_count', 'overstatement']

for blockno, block_data in isps_per_block_cov_noncov_only_0.items():
    if blockno not in urban_rural_dict:
        continue
    block_class = urban_rural_dict[blockno]
    state_fips = int(blockno / 10**13)
    state = VALID_STATE_FIPS[state_fips]
    
    fcc_data = block_data["fcc"]
    tool_data = block_data["tool"]
    fcc_count_avg = sum(fcc_data)/len(fcc_data)
    tool_count_avg = sum(tool_data)/len(tool_data)

    competition_state_stats[state].append([state, blockno, 0, tool_count_avg, fcc_count_avg, tool_count_avg / fcc_count_avg])
    if blockno in isps_per_block_cov_noncov_only_25:
        competition_state_stats[state].append([state, blockno, 25, tool_count_avg, fcc_count_avg, tool_count_avg / fcc_count_avg])

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=3, figsize=(10,10), sharey=True)

for i, state in enumerate(STATES):
    ax_row = int(i / (len(STATES)/3))
    ax_col = int(i % (len(STATES)/3))
    df_counts_per_block = pd.DataFrame(competition_state_stats[state], columns = column_names)
    ax = sns.boxplot(x = 'speed', y = 'overstatement', data = df_counts_per_block,fliersize=0,ax=axes[ax_row][ax_col], order=[0, 25]) 
    if ax_col == 0 and ax_row != 1:
        ax.set_ylabel("Proportion of # of Providers Per Block (BATs)")
    else:
        ax.set_ylabel("")
        #ax.get_legend().remove()
    ax.set_xlabel("")
    ax.set_title("{}".format(LABELS[state]))
    ax.grid(False)
    
fig.tight_layout()
fig.savefig("fig9.pdf")

In [ ]:
for code, state in VALID_STATE_FIPS.items():
    state_data = all_state_data[state]
    lol = (state_data[(state_data.addr_census_block / 10**13).astype(int) != code])
    for col in lol.columns[11:]:
        print(lol[col].unique())

# Appendix

## Figure 7 (Overstatements by Min Speed)

In [ ]:
speed_isps = ['att', 'centurylink', 'consolidated', 'windstream']
all_isp_speed_max = defaultdict(list)

for state in all_state_data:
    state_data_global = all_state_data[state]
    
    isps = [x for x in speed_isps if x in ISPS_PER_STATE[state]]
    
    for isp in isps:
        state_isp_data_cols = data_cols[state][isp]
        fcc_cols = state_isp_data_cols["fcc"]
        fcc_speed_cols = state_isp_data_cols["fcc_speed"]
        tool_cols = state_isp_data_cols["tool"]
        tool_speed_cols = state_isp_data_cols["tool_speed"]
        timestamp_cols = state_isp_data_cols["timestamp"]
        
        state_data = state_data_global.copy(deep=True)
        
        state_data = state_data[((state_data[timestamp_cols] < PAPER_TIME)).any(axis=1)]
        
        state_data["FCC"] = state_data[fcc_speed_cols].max(axis=1)
        state_data["BATs"] = state_data[tool_speed_cols].max(axis=1)
        
        all_isp_speed_max[isp].append(state_data[["addr_id", "addr_census_block", "FCC", "BATs"]])
        
for isp, isp_speed_data in list(all_isp_speed_max.items()):
    all_isp_speed_max[isp] = pd.concat(isp_speed_data).dropna()

In [ ]:
speed_isps = ['att', 'centurylink', 'consolidated', 'windstream']
fig7_data = defaultdict(lambda: defaultdict(Counter)) # speed -> {blockno -> {'fcc', 'tool'}}

for i, (isp, isp_speed_data) in enumerate(sorted(all_isp_speed_max.items(), key=lambda x: speed_isps.index(x[0]))):
    print(isp)
    isp_speed_data = isp_speed_data.copy(deep=True)
    isp_speed_data = isp_speed_data[isp_speed_data["BATs"] > 0]
    
    for address in isp_speed_data.itertuples():
        blockno = getattr(address, "addr_census_block")
        fcc_speed = getattr(address, "FCC")
        tool_speed = getattr(address, "BATs")
        
        good = False
        if tool_speed >= fcc_speed:
            good = True
        
        for speed in [0, 25, 50, 100, 200]:
            if fcc_speed >= speed:
                fig7_data[speed][blockno]['fcc'] += 1
                if good:
                    fig7_data[speed][blockno]['tool'] += 1
    
total_line = []
rural_line = []
urban_line = []

for speed, speed_data in fig7_data.items():
    rural_count = []
    urban_count = []
    total_count = []
    
    for blockno, block_data in speed_data.items():
        block_class = urban_rural_dict[blockno]
        if block_class == "R":
            rural_count.append(block_data["tool"]/block_data["fcc"])
        else:
            urban_count.append(block_data["tool"]/block_data["fcc"])
        total_count.append(block_data["tool"]/block_data["fcc"])
    
    rural_count = sum(rural_count)/len(rural_count)
    urban_count = sum(urban_count)/len(urban_count)
    total_count = sum(total_count)/len(total_count)
    
    rural_line.append(rural_count)
    urban_line.append(urban_count)

    total_line.append(total_count)

In [ ]:
ticks = [0, 1, 2, 3, 4]
labels = ["0", "25", "50", "100", "200"]
fig, ax = plt.subplots(1,1)
plt.plot(rural_line, marker='o', label='Rural')
plt.plot(urban_line, marker='o', label='Urban')
plt.plot(total_line, marker='o', label='Total')
ax.set_xticks(ticks)
ax.set_xticklabels(labels)
plt.legend()
plt.ylabel("")
plt.xlabel("Minimum Download Speed (Mbps)")
# ax.set_yticklabels(['{:,.0%}'.format(x) for x in ticks])

fig.tight_layout()
plt.savefig("overstatements_lower_speeds.pdf")